# Unsupervised Learning 
> Kohonen Network (Hamming Network + Maxnet), and PCA Network

Unsupervised learning using pure python + numpy.


In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris

In [2]:
data = load_iris()
x = data.data
x = (x - x.mean(axis=0)) / x.std(axis=0)
y = data.target
data.feature_names, data.target_names

(['sepal length (cm)',
  'sepal width (cm)',
  'petal length (cm)',
  'petal width (cm)'],
 array(['setosa', 'versicolor', 'virginica'], dtype='<U10'))

In [3]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=3)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((120, 4), (30, 4), (120,), (30,))

In [4]:
def evaluator(y_test, y_pred):
    conf_matrix = np.zeros((3, 3))
    for testi, predi in zip(y_test, y_pred):
        conf_matrix[testi][predi] += 1

    print("Confusion matrix:")
    print(conf_matrix)
    print()

    class_counts = {}
    for i in range(3):
        tp = conf_matrix[i][i]
        tn = np.sum(np.delete(np.delete(conf_matrix, i, 0), i, 1))
        fp = np.sum(np.delete(conf_matrix[:, i], i, 0))
        fn = np.sum(np.delete(conf_matrix, i, 1)[i, :])

        class_counts[i] = {"tp": tp, "tn": tn, "fp": fp, "fn": fn}

    print("Class Accuracy Precision Recall F1-score")
    for label, counts in class_counts.items():
        tp = counts["tp"]
        tn = counts["tn"]
        fp = counts["fp"]
        fn = counts["fn"]
        acc = (tp + tn) / (tp + tn + fp + fn)
        prec = tp / (tp + fp)
        rec = tp / (tp + fn)
        f1 = 0 if ((prec + rec) == 0) else 2 * ((prec * rec) / (prec + rec))
        print(f"{label:>3d}\t {acc:>4.3f}\t {prec:>4.3f}\t {rec:>4.3f}\t {f1:>4.3f}")

    print(f"\nOverall accuracy: {np.sum(y_pred == y_test) / len(y_pred)}\n")

In [5]:
# baseline model (K-Means) for comparison
from sklearn.cluster import KMeans

km = KMeans(n_clusters=3)
km.fit(x_train)
print("Train:")
y_pred = km.predict(x_train)
evaluator(y_train, y_pred)
print("Test:")
y_pred = km.predict(x_test)
evaluator(y_test, y_pred)

Train:
Confusion matrix:
[[ 0. 40.  0.]
 [32.  0.  8.]
 [14.  0. 26.]]

Class Accuracy Precision Recall F1-score
  0	 0.283	 0.000	 0.000	 0.000
  1	 0.333	 0.000	 0.000	 0.000
  2	 0.817	 0.765	 0.650	 0.703

Overall accuracy: 0.21666666666666667

Test:
Confusion matrix:
[[ 0. 10.  0.]
 [ 7.  0.  3.]
 [ 3.  0.  7.]]

Class Accuracy Precision Recall F1-score
  0	 0.333	 0.000	 0.000	 0.000
  1	 0.333	 0.000	 0.000	 0.000
  2	 0.800	 0.700	 0.700	 0.700

Overall accuracy: 0.23333333333333334



## Kohonen Network
A Hamming network with a Maxnet on top

In [6]:
class Kohonen(object):
    def __init__(self, num_clusters):
        self.P = num_clusters

    def maxnet(self, x):
        e = -1 / self.P
        while True:  # run until a winner is found
            # calculate the next node values
            new = np.zeros(self.P)
            for i in range(self.P):
                new[i] = max(0, x[i] + e * np.sum(np.delete(x, i)))
            x = new
            if not np.any(x):
                # in edge cases when all nodes = 0, return None
                return None
            winners = np.where(x > 0)[0]
            if len(winners) == 1:
                return winners[0]

    def train(
        self,
        x,
        y,
        x_test,
        y_test,
        init_lr=0.5,
        lr_dec=(lambda l: l / 2),
        threshold=0.001,
        verbose=True,
    ):
        n_train, input_size = x.shape
        n_test = x_test.shape[0]
        self.lr = init_lr
        # initialize centroids as random data points from their respective class
        self.W = np.zeros((self.P, input_size))
        for i in range(self.P):
            w = x[np.random.choice(np.argwhere(y == i).flatten())]
            self.W[i] = w
        self.hamming_b = -input_size / 2

        # training loop
        done = False
        while not done:
            # loop over input data until halting criteria is met
            for i, xi in enumerate(x):
                # Hamming net feedforward
                hamming_o = self.W @ xi - self.hamming_b
                j = self.maxnet(hamming_o)  # winning output node
                if j is None:
                    # Skip weight update if no winning node is found (very rare)
                    if verbose:
                        print("No winning node for input", xi)
                    continue
                # Calculate update for the winning node
                delta_wj = self.lr * (xi - self.W[j])
                # Stop if the update's magnitude is below the threshold
                if np.sqrt(delta_wj @ delta_wj) < threshold:
                    done = True
                    break
                else:
                    # Apply the weight update
                    self.W[j] += delta_wj
                    # Decrease the learning rate using the decrease function
                    self.lr -= lr_dec(self.lr)

                # calculate the metrics for the iteration
                train_acc = (self.predict(x) == y).sum() / n_train
                test_acc = (self.predict(x_test) == y_test).sum() / n_test
                if verbose:
                    print(
                        f"iteration: {i+1}, winning node: {j}, train acc: {train_acc:.4f}, test acc: {test_acc:.4f}"
                    )

    def predict(self, x):
        # feedforward and predict using learned weights
        y = np.zeros(x.shape[0], int)
        for i, xi in enumerate(x):
            hamming_o = self.W @ xi - self.hamming_b
            pred = self.maxnet(hamming_o)
            if pred is None:
                # In the rare edge case when maxnet can't find a max,
                # make a random prediction
                pred = np.random.randint(3)
            y[i] = pred
        return y.T

In [7]:
# train and test the Kohonen Network
kohonen = Kohonen(3)
kohonen.train(
    x_train,
    y_train,
    x_test,
    y_test,
    init_lr=0.5,
    lr_dec=(lambda l: l / 2),
    threshold=0.001,
    verbose=False,
)
print("Train:")
y_pred = kohonen.predict(x_train)
evaluator(y_train, y_pred)
print("Test:")
y_pred = kohonen.predict(x_test)
evaluator(y_test, y_pred)

Train:
Confusion matrix:
[[40.  0.  0.]
 [ 3. 17. 20.]
 [ 0.  2. 38.]]

Class Accuracy Precision Recall F1-score
  0	 0.975	 0.930	 1.000	 0.964
  1	 0.792	 0.895	 0.425	 0.576
  2	 0.817	 0.655	 0.950	 0.776

Overall accuracy: 0.7916666666666666

Test:
Confusion matrix:
[[10.  0.  0.]
 [ 1.  5.  4.]
 [ 0.  0. 10.]]

Class Accuracy Precision Recall F1-score
  0	 0.967	 0.909	 1.000	 0.952
  1	 0.833	 1.000	 0.500	 0.667
  2	 0.867	 0.714	 1.000	 0.833

Overall accuracy: 0.8333333333333334



## Principle Component Analysis Network
Approximates PCA

In [8]:
class PCA(object):
    def __init__(self, dims):
        self.dims = dims

    def train(self, x, lr=0.001, epochs=10):
        n_train, input_size = x.shape
        n_test = x_test.shape[0]
        self.W = np.random.randn(input_size, self.dims) * 0.01
        self.b = np.zeros(1)

        for _ in range(epochs):
            # feedforward
            y = x @ self.W

            # update weights
            self.W += lr * (x - y @ self.W.T).T @ y

    def predict(self, x):
        return x @ self.W

In [9]:
# train the PCA model and obtain the new train and test datasets
pca = PCA(3)
pca.train(x_train, epochs=100)
x_train_PCA = pca.predict(x_train)
x_test_PCA = pca.predict(x_test)
print("Final weight is:\n", pca.W)

Final weight is:
 [[ 0.61303574  0.05261625 -0.13632559]
 [ 0.38609266 -0.87498523 -0.11864248]
 [ 0.45139119  0.35384083 -0.13385725]
 [ 0.43055992  0.32565808 -0.20752721]]


In [10]:
# train and test the Kohonen Network with the new datasets
kohonen_PCA = Kohonen(3)
kohonen_PCA.train(
    x_train_PCA,
    y_train,
    x_test_PCA,
    y_test,
    init_lr=0.5,
    lr_dec=(lambda l: l / 2),
    threshold=0.001,
    verbose=False,
)
print("Train:")
y_pred = kohonen_PCA.predict(x_train_PCA)
evaluator(y_train, y_pred)
print("Test:")
y_pred = kohonen_PCA.predict(x_test_PCA)
evaluator(y_test, y_pred)

Train:
Confusion matrix:
[[40.  0.  0.]
 [ 0. 19. 21.]
 [ 0.  2. 38.]]

Class Accuracy Precision Recall F1-score
  0	 1.000	 1.000	 1.000	 1.000
  1	 0.808	 0.905	 0.475	 0.623
  2	 0.808	 0.644	 0.950	 0.768

Overall accuracy: 0.8083333333333333

Test:
Confusion matrix:
[[10.  0.  0.]
 [ 0.  6.  4.]
 [ 0.  0. 10.]]

Class Accuracy Precision Recall F1-score
  0	 1.000	 1.000	 1.000	 1.000
  1	 0.867	 1.000	 0.600	 0.750
  2	 0.867	 0.714	 1.000	 0.833

Overall accuracy: 0.8666666666666667

